In [27]:
import torch
import torch.nn as nn
import torchvision
import torch.utils.data
import torchvision.transforms as transforms
from tqdm import tqdm

In [28]:
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

1
GeForce GTX 1080


In [29]:
BATCH_SIZE=32
NUM_WORKERS=4
EPOCHS=100

In [30]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        (0.4914, 0.48216, 0.44653),
        (0.24703, 0.24349, 0.26159)
    )])

In [31]:
# trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
# testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [32]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

In [33]:
print(trainloader.dataset.data.shape)
print(testloader.dataset.data.shape)

(50000, 32, 32, 3)
(10000, 32, 32, 3)


In [34]:
# class AlexNet(nn.Module):
#     def __init__(self, num_classes=10):
#         super(AlexNet, self).__init__()
#         self.conv1 = nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2)
#         self.relu = nn.ReLU(inplace=True)
#         self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2)
#         self.conv2 = nn.Conv2d(64, 192, kernel_size=5, padding=2)
#         self.conv3 = nn.Conv2d(192, 384, kernel_size=3, padding=1)
#         self.conv4 = nn.Conv2d(384, 256, kernel_size=3, padding=1)
#         self.conv5 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
#         self.avgpool = nn.AdaptiveAvgPool2d((6,6))
#         self.fc1 = nn.Linear(256 * 6 * 6, 4096)
#         self.fc2 = nn.Linear(4096, 4096)
#         self.fc3 = nn.Linear(4096, num_classes)
    
#     def forward(self, x):
#         x = self.relu(self.conv1(x))
#         x = self.maxpool(x)
#         x = self.relu(self.conv2(x))
#         x = self.maxpool(x)
#         x = self.relu(self.conv3(x))
#         x = self.relu(self.conv4(x))
#         x = self.relu(self.conv5(x))
#         x = self.maxpool(x)
#         x = self.avgpool(x)
#         x = x.view(x.size(0), 256 * 6 * 6)
#         x = self.relu(self.fc1(x))
#         x = self.relu(self.fc2(x))
#         return self.fc3(x)

In [35]:
# net = AlexNet()
# net.to(device)
# criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(net.parameters(), lr=3e-4)

In [37]:
class Net(nn.Module):
    def __init__(self, num_classes=10):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2,2)
        self.fc = nn.Linear(128 * 4 * 4, num_classes) # 128 is output from conv3, 4*4 is because 3x pooling reduce input size 32 by factor of 2 for 3 times (i.e. 32/2 = 16/2 = 8/2 = 4)
    
    def forward(self, x):
        x = self.conv1(x)
        x = nn.functional.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = nn.functional.relu(x)
        x = self.pool(x)
        x = self.conv3(x)
        x = nn.functional.relu(x)
        x = self.pool(x)
        x = x.view(-1, 128 * 4 * 4)
        return self.fc(x)


In [38]:
net = Net()
net.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=3e-4)

In [40]:
for epoch in tqdm(range(EPOCHS)):
    for i, data in enumerate(trainloader, 0):
        # Get inputs
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        # Set optimizer gradient to zero (required for pytorch)
        optimizer.zero_grad()
        # Foward pass -> backward pass -> optimize
        output = net(inputs)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()

100%|██████████| 100/100 [07:23<00:00,  4.44s/it]


In [41]:
correct, total = 0, 0
predictions = []
net.eval()

Net(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc): Linear(in_features=2048, out_features=10, bias=True)
)

In [43]:
for i, data in tqdm(enumerate(testloader, 0)):
    inputs, labels = data
    inputs, labels = inputs.to(device), labels.to(device)
    outputs = net(inputs)
    _, predicted = torch.max(outputs.data, 1)
    predictions.append(outputs)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

313it [00:00, 435.04it/s]


In [44]:
print(f'Testset acc: {100 * correct / total}')

Testset acc: 73.5944976076555
